In [1]:
import pandas as pd 
import plotly.express as px
import pygal as pg
from string import Template
from IPython.core.display import display, HTML
from IPython.display import HTML, Javascript, display

neo4j_url=%env NEO4J_URL

%reload_ext cypher
%config CypherMagic.uri=neo4j_url + "/db/data"

def configure_d3():
    """Tell require where to get d3 from in `require(['d3'])`"""
    display(Javascript("""
    require.config({ 
      paths: {
        lodash: "/notebooks/vis/lib/lodash.min",  
        d3: "/notebooks/vis/lib/d3.v4.min"
      }
    })"""))

configure_d3()

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="/notebooks/vis/lib/svg.jquery.js"></script>
  <script type="text/javascript" src="/notebooks/vis/lib/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [2]:
%cypher MATCH (n) RETURN count(n)

In [2]:
packageHierarchy = %cypher MATCH (p:Package)-[:CONTAINS*]->(t:Type) \
                           WITH DISTINCT p.fqn AS packageName \
                           MATCH (p:Package{fqn: packageName})-[:CONTAINS]->(child) \
                           WHERE (child:Package AND exists((child)-[:CONTAINS*]->(:Type))) OR child:Type \
                           WITH p, child, child:Type AS leaf \
                           RETURN DISTINCT p.fqn AS Parent_Fqn, p.name AS Parent_Name, child.fqn AS Child_Fqn, child.name AS Child_Name, leaf AS Child_Is_Leaf \
                           ORDER BY Parent_Fqn

package_hierarchy_df = packageHierarchy.get_dataframe()

In [3]:
text = Template(open('../vis/circle-packing/circle-packing-diagram.html', 'r').read().replace("\n","")).substitute({
    'circle_data': package_hierarchy_df.to_csv(index = False).replace("\r\n","\\n").replace("\n","\\n"),
    'container': 'type-packing-diagram'
})

HTML(text)

In [4]:
moduleDependencies = %cypher MATCH (p1:Package)-[:CONTAINS]->(t1:Type), \
                                   (p2:Package)-[:CONTAINS]->(t2:Type), \
                                   (t1)-[dep:DEPENDS_ON]->(t2) \
                             WHERE p1 <> p2 \
                             RETURN p1.name AS Source, \
                                    p2.name AS Target, \
                                    COUNT(dep) AS X_Count \
                             ORDER BY Source DESC

moduleDependenciesData = moduleDependencies.get_dataframe().to_csv(index = False).replace("\r\n","\\n").replace("\n","\\n")

In [5]:
text = Template(open('../vis/chord/chord-diagram.html', 'r').read().replace("\n","")).substitute({
    'chord_data': moduleDependenciesData, 
    'container': 'module-chord-diagram'})

HTML(text)